In [1]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
import modules as mm
import numpy as np

In [2]:

def softmax(x):
    exps = mm.exp(x-x.data.max(axis=1, keepdims=True))
    return exps/exps.sum(axis=1, keepdims=True)

def NLL(y_pred, y_true, reduce=True):
    loss = mm.log(y_pred[np.arange(len(y_true)), y_true.astype(int)])
    return -loss.mean() if reduce else -loss

def NLL_np(y_pred, y_true):
    """
    NLL loss for numpy arrays.
    """
    loss = np.log(y_pred[np.arange(len(y_true)), y_true.astype(int)])
    return -loss


class Linear(mm.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = mm.Parameter(
            data=np.random.uniform(-1, 1, (in_features, out_features)) 
                * np.sqrt(6 / (in_features + out_features)),
            dtype=np.float32)
        self.bias = mm.Parameter(
            data=np.zeros((1, out_features)),
            dtype=np.float32,
            regularize=False)

    def forward(self, x):
        return x @ self.weight + self.bias


class ANN(mm.Module):
    def __init__(self, units=[]):
        super().__init__()
        self.units = units
        self.layers = []
        for i in range(len(units)-1):
            self.layers.append(Linear(units[i], units[i+1]))
        self.register_layers(self.layers)

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = mm.sigmoid(layer(x))
        x = self.layers[-1](x)
        return x

class ANNSoftmax(mm.Module):
    def __init__(self, input_dim, n_classes, units):
        super().__init__()
        self.units = units
        self.n_classes = n_classes
        self.ann = ANN((input_dim,) + tuple(units) + (n_classes-1,))

    def forward(self, x):
        x = self.ann(x)
        out = mm.zeros((x.shape[0], x.shape[1]+1), dtype=x.data.dtype)
        out[:, :-1] = x
        out = softmax(out)
        return out

In [3]:
import pandas as pd
import evaluation as eval

def accuracy(y_pred, y_true):
    y_pred = np.argmax(y_pred.data, axis=1)
    return y_pred == y_true

In [4]:
np.random.seed(3)

# import and prepare data
data_path = 'housing3.csv'
data = pd.read_csv(data_path)
X = data.drop(columns='Class').to_numpy()
m = X.mean(axis=0)
s = X.std(axis=0)
X = (X - m) / s
y = pd.factorize(data['Class'])[0]
n_classes = len(np.unique(y))

In [5]:
# define and train simple multinomial logistic regression model
learner = mm.Learner(
    model=ANNSoftmax,
    model_args={
        'input_dim': X.shape[1],
        'n_classes': n_classes,
        'units': []
    },
    loss=NLL,
    lambda_=0.0
)
metrics = eval.k_fold(X, y, learner.fit, NLL_np, k=5)
score = eval.bootstrap(metrics, reduce=np.mean, bootstraps=10000)
display(f'Accuracy: {round(score[0], 3)} +- {round(score[1], 3)}')

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL


'Accuracy: 0.28 +- 0.028'

In [6]:
# define and train a bit more complex model
learner = mm.Learner(
    model=ANNSoftmax,
    model_args={
        'input_dim': X.shape[1],
        'n_classes': n_classes,
        'units': [42]
    },
    loss=NLL,
    lambda_=0.004
)
metrics = eval.k_fold(X, y, learner.fit, NLL_np, k=5)
score = eval.bootstrap(metrics, reduce=np.mean, bootstraps=10000)
display(f'Accuracy: {round(score[0], 3)} +- {round(score[1], 3)}')

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


'Accuracy: 0.255 +- 0.024'